## Data Preprocessing - 1
##### this script will work with simple data preprocessing steps to clean the text data
###### we will use youtube comments to use different ways to data preprocessing methodology

In [9]:
import nltk

ModuleNotFoundError: No module named 'nltk'

In [1]:
## importing required libraries

import os
import pandas as pd
import numpy as np

In [3]:
### reading youtube comment data

utube_df = pd.read_csv('../Data/YoutubeCommentsDataSet.csv')
utube_df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [4]:
df_corpus = utube_df.Comment.to_xarray()

ImportError: Missing optional dependency 'xarray'.  Use pip or conda to install xarray.